In [19]:
# Restricted Boltzmann machine classical
# Classical RBM libraries
import numpy as np
import pandas as pd
from math import pi, acos, sin, sqrt
#from scipy.linalg import norm
from numpy.linalg import norm
import matplotlib.pyplot as plt

# Quantum computing RBM libraries
import qiskit
from qiskit import transpile
from qiskit import Aer, execute, QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.primitives import Sampler
from qiskit.providers.fake_provider import FakeManilaV2
from qiskit.visualization import plot_histogram, plot_state_city
from qiskit.circuit import QuantumCircuit, ParameterVector
from qiskit.circuit.library import RYGate
import qiskit.quantum_info as qi
from qiskit.quantum_info import Statevector, DensityMatrix


# Defining RBM class
class RBM:
    def __init__(self, num_vis, num_hid): 
        self.num_vis = num_vis
        self.num_hid = num_hid
        self.learning_rate = 0.1
        
        # Initial weights and biases
        # Network weights (W(n,m))
        self.W = np.random.normal(0, 0.1, (num_vis, num_hid) )
        self.W_ang = self.W
        
        # self.a and self.b will store the amplitude of qi along |1> state of being bit string 1 
        # Visible bias
        self.a = np.zeros(num_vis) # states probability
        self.a_ang = np.zeros(num_vis)
        # Reference vector for angle, the best superposition to project vectors
        self.a_ref = np.ones(self.num_vis)
        self.a_ref = self.a_ref/norm(self.a_ref)
        # Alternative reference vector for visual as |0000...0_n> = |phi_0> state
        # self.a_ang_ref = np.zeros(self.num_vis)
        # self.a_ang_ref[0] = 1 
        # Probability vectors to |psi_0> for num_vis qubits
        self.a_vecs = np.zeros((num_vis, 2)) 
        for i in range(num_vis):
            self.a_vecs[i] = [1,0] 
        
        # Hidden bias
        self.b = np.zeros(num_hid)
        self.b_ang = np.zeros(num_hid)
        # Reference vector for angle, the best superposition to project vectors
        self.b_ref = np.ones(self.num_hid)
        self.b_ref = self.b_ref/norm(self.b_ref)
        # Alternative reference vector for hidden as |0000...0_n> = |phi_0> state
        # self.b_ang_ref = np.zeros(self.num_vis)
        # self.b_ang_ref[0] = 1 
        # Probability vectors to |psi_0> num_hid qubits
        self.b_vecs = np.zeros((num_hid, 2))
        for i in range(num_hid):
            self.b_vecs[i] = [1,0] 
        
    def sigmoid(self, x):
        ''' Sigmoid logistic is utilized to approximate 
            the complicated energy sampling (integrals) '''
        return 1.0/( 1.0 + np.exp(-x) )
        
        
    def gibbs_sampling(self, vis_prob):
        ''' Gibbs sampling is a conditional probability P(v=1|h) or P(h=1|b)
         for sampling the energetic description
         INPUT:
         vis_prob =======> measurment/data into the sampler (can be binary or not)
         OUTPUT:
         hid_prob =======> hidden probability states
         hid_states =====> hidden states (0 or 1)
         vis_prob_rec ===> visual probability states
         '''
        # <v,W> =vpw [size m]
        vpw = np.dot(vis_prob, self.W)
        
        # S(<v,W> + b) [size m]
        hid_prob = self.sigmoid(vpw + self.b)
        
        # the hid_prob serves as an activation of (0,1) states
        hid_states = np.random.rand(self.num_hid) < hid_prob 
        
        # Visible reconstruction according to previous states
        
        # <W,h> = <h,W^T> = whp [size n]
        whp = np.dot(hid_states, self.W.T)
        
        # S(<v,W> + b) [size n]
        vis_prob_rec = self.sigmoid( whp + self.a )
        
        return hid_prob, hid_states, vis_prob_rec
    
        
    def get_angle(self, v_in, method='projection', ref_vec='+') :
        '''  This function obtains converts vector information 
             into angle for encrypting classical information to
             quantum information style.
         
             INPUT: 
             mode  =======> 0 uses projection into |psi_0>| 1 uses exponential to convert it
             v_in  =======> desired quantum state's (vector) angle
             TODO:
             v0_in =======> this can be tunned but meant to be |psi_0> (very initial quantum state/vector)
         
             OUTPUT:
             proj ========> array of projected quantum states in to reference quantum state 
                            ang = <psi_k | psi_0>/ ( norm(psi_k) * norm(psi_0) )
                            in practice norm(psi_k) has to be 1 given that are in unit circle description
        '''
        # Input amplitude of |1> state
        v_in = np.squeeze( np.asarray(v_in) )
        vshape = v_in.shape
        n = vshape[0]
        
        # I stored pointing toward |1> state in get_statesNprobs
        vprobs = np.zeros((n,2))
        for i in range(n): 
            ang0 = acos(v_in[i])
            # Reconstructing vector state for i-th qubit
            vprobs[i,:] = [sin(ang0), v_in[i]]
        
        # "Basis" or vector to project qubit
        if ref_vec == '0': 
            # |0> state
            v0 = np.array((1,0))
        elif ref_vec == '1':
            # |1> state 
            v0 = np.array((0,1))
        elif ref_vec == '-':
            # |-> state
            v0 = np.array((1,-1))/sqrt(2)
        else :
            # |+> state 
            # Hadamard transformation most even probability projection
            v0 = np.array((1,1))/sqrt(2)
   
        proj = np.zeros(n)
        if method == 'projection' :
            # Angle projection into |v0> state
            for iq in range(n) :
                proj[iq] = np.dot( vprobs[iq,:] , v0) # project to |0> basis
                
        #elif mode == 'exps' : 
            # Angle defined in terms of exponents
            #proj[iq] = exp(np.dot(v,)
        
        elif mode == 'hpi' :
            for i in range(n):
                proj[iq] = pi/2

        #else
            # Not defined
    
        return proj
    
    
    def get_statesNprobs(self, quantum_state, n, rho):
        ''' This will write back binary string to hidden and visual states 
            will provide the probabilities associated to the given states
        '''
        nq = sum(n) + n[0]*n[1]
        vis_state = np.zeros(n[0])
        vis_prob  = np.zeros(n[0])
        hid_state = np.zeros(n[1])
        hid_prob  = np.zeros(n[1])

        # Probabilities of measuring only qubit-i in a given state
        qprobs = np.zeros((nq,2))
        cprobs = np.zeros((nq))
        for iq in range(nq):
            # Passing quantum state probabilities from |1> state
            qprobs[iq] = rho.probabilities([iq])
            print('Qubit-{} probs: {}'.format(iq, qprobs[iq]))
            cprobs[iq] = qprobs[iq,1]
            
            #if quantum_state[iq] == "0":
                # Probably only assing the state one probability only?
                # this is equivalent to <0|q_i> = qprob_i in state |0>
                #cprobs[iq] = qprobs[iq,0]
            #else:
                # this is equivalent to <1|q_i> = qprob_i in state |1>
                #cprobs[iq] = qprobs[iq,1]
                
        # Obtaining the visual and hidden states "binary" nad probabilities 
        # alligned to |1>         
        
        jq = -1
        for iq in range(nq-1,-1,-1): 
            
            print('iq in state probs : ', iq)
            jq = jq + 1 
            if (iq > nq - n[0] - 1) & (iq <= nq): 
                vis_state[jq] = int(quantum_state[iq])  
                # Probability vector pointing toward |1>
                #vis_prob[jq]  = sqrt(qprobs[iq]) 
                # probability vector amplitud pointing toward |1> and binarize
                vis_prob[jq]  = sqrt(cprobs[iq])*vis_state[jq]
                print('Visible : '+str(vis_state[jq]) + " " + str(vis_prob[jq]) )
                
            elif (iq > nq -sum(n)) & (iq <= nq - n[0]):
                hid_state[jq-n[0]] = int(quantum_state[iq])
                # Probability vector pointing toward |1>
                #hid_prob[jq-n[0]]  = sqrt(qprobs[iq]) 
                # probability vector amplitud pointing toward |1> and binarize
                hid_prob[jq-n[0]]  = sqrt(cprobs[iq])*hid_state[jq-n[0]]
                print('Hidden : '+str(hid_state[jq-n[0]]) + " " + str(hid_prob[jq-n[0]]) )

                
                
        return vis_state, hid_state, vis_prob, hid_prob
    
        
    def qc_gibbs_sampling(self, vis_prob, nshots) :
        ''' qc_gibbs_sampling is build to provide Gibbs sampling through 
            quantum computing in form of zeros and ones for a 
            restricted Boltzmann machine (RBM).
            INPUT:
            vis_prob =====> measurment/data (has to be binarized)
            nshots  =====> number of times to run quantum circuit 
            OUTPUT:
            hid_prob =====> hidden layer probabilities can be the vector form of state 
                            <phi_k | Psi> where k spans from 1 to n described by visual layer qubits
                            <phi_k | Psi> = vis_prob[k]
            hid_states ===> hidden layer states (measurement in qc)
            vis_prob_rec => visual layer probabilities can be the vector form of state 
                            <chi_k | Psi> where k spans from 1 to m described by visual layer qubits 
                            <chi_k | Psi> = hid_prob[k]
            NOTE: 
            ???I'll use analogously a, b and W activations but in radians (testing purpouse) 
        '''

        n = [self.num_vis, self.num_hid] 
      
        # Number of qubits, ancilla gate operations are num_vis*num_hid
        nq = sum(n) + n[0]*n[1]

        # Start the circuit with nq size
        circ = qiskit.QuantumCircuit(nq)
    
        # Initial state of visual nodes
        self.a_ang = self.get_angle(self.a, method='projection', ref_vec='+')
        for iq in range(n[0]) :
            circ.ry(self.a_ang[iq],iq)

        # Initial state of hidden nodes
        self.b_ang = self.get_angle(self.b, method='projection', ref_vec='+')
        for jq in range(n[1]) :
            circ.ry(self.b_ang[jq],n[0]+jq)

        mq = sum(n) - 1
        for iq in range(n[0]) :
            # do angle transform of W here?? may keep it as angle all along 
            #self.W_ang[iq,:] = self.get_angle(self.W[iq,:], 0)
            for jq in range(n[1]) :
        
                mq = mq + 1
                ccry = RYGate(self.W_ang[iq,jq]).control(2)
                
                # Controlling iq -> visual layer, jq -> hidden layer, mq -> worker qubit
                circ.append(ccry, [iq, n[0]+jq, mq ])    
       
        # Building density matrix
        rho = DensityMatrix(circ)
    
        # Run the quantum circuit on a statevector simulator backend
        #sim_mode = 'qasm_simulator'
        sim_mode = 'aer_simulator'

        simulator = Aer.get_backend(sim_mode)

        # Transpile reshapes the circuit to match quantum device topology (optimize circuit)
        circ = transpile(circ, simulator)

        # Run and get counts format #1 Aer
        circ.measure_all()

        result = simulator.run(circ,shots=nshots).result()
        counts = result.get_counts(circ)
        #print(counts)

        # Access all the states and counts
        #count_keys=list(counts.keys())
        #ikey=count_keys[0]
        #counts.get(ikey)

        # Most frequent state and probability
        most_freq = counts.most_frequent()
        #most_freq_prob = counts.get(most_freq)/nshots

        # Obtaining states and probabilities of visual and hidden nodes
        vis_states, hid_states, vis_prob_rec, hid_prob = self.get_statesNprobs(most_freq, n, rho)
        
        # This is different because we construct both together and
        # the probability comes from the whole probability state
        return hid_prob, hid_states, vis_prob_rec, vis_states


        
    def train(self, data, mod_comp, num_epochs=100) : 
        ''' Training RBM according to measurment (vis_prob) 
            NOTE: vis_prob can be binary or not'''
        
        for epoch in range(num_epochs) :
            k = 0
            for v0 in data:
                k = k + 1
                # Optimization is carried out with contrastive divergence (CD)
                if mod_comp == 'classical':
                    # Positive phase of sampling
                    hid_prob0, hid_states0, vis_prob1 = self.gibbs_sampling(v0)               
               
                    # Negative phase of sampling
                    hid_prob1, hid_states1, vis_prob2 = self.gibbs_sampling(vis_prob1)
                    
                elif mod_comp == 'qc':
                    # both random states, although probability vectors do not change here).
                    nshots = 1000
                    # Positive phase of sampling
                    hid_prob0, hid_states0, vis_prob1, vis_state1 = self.qc_gibbs_sampling(v0, nshots)               

                    # Negative phase of sampling ( A single run in quantum computing produces 
                    #hid_prob1, hid_states1, vis_prob2 = self.qc_gibbs_sampling(vis_prob1, nshots)
                    hid_prob1, hid_states1, vis_prob2, vis_state2 = self.qc_gibbs_sampling(vis_state1, nshots)
                    
                    print("finished epoch number ..." +str(epoch) +" data inx "+str(k))
                    
                else :
                    print("Mode "+ mod_comp +" not supported ")
                    print("Please select either qc or classical")
                    break
                    
                # Update weights and biases with CD approach
                # W = eps ( v0 (x) h0 - v1 (X) h1)
                self.W += self.learning_rate*(np.outer(v0, hid_prob0) - np.outer(vis_prob1, hid_prob1) )
                
                # Updating visible bias according learning rate
                self.a += self.learning_rate*( v0 - vis_prob1)
                
                # Updating hidden bias according learning rate
                self.b += self.learning_rate*( hid_prob0 - hid_prob1)
        
        
        
    def generate_sample(self, num_samples): 
        # Provide a sample according the learned W, a and b for visible and hidden layers
        samples = np.zeros( (num_samples, self.num_vis) )
        
        for i in range(num_samples):
            
            # Choose random probabilitie to stabilize
            vis_prob = np.random.rand(self.num_vis)
            # Gibbs sampling to reach equillibrium
            for _ in range(1000):
                _, _, vis_prob = self.gibbs_sampling(vis_prob)
                samples[i] = vis_prob
                
            
        return samples
                

# Generating binary data for test case
num_samples = 1
num_vis = 2 
num_hid = 2
binary = True

data = np.random.randint(2, size = (num_samples, num_vis) ) 

# Create and train the RBM 
rbm = RBM(num_vis, num_hid) 
#rbm.train(data, mod_comp = 'classical', num_epochs = 100)
rbm.train(data, mod_comp = 'qc', num_epochs = 1)

print("W final learn : " + str(rbm.W) )
print("Vis_bias final learn : " + str(rbm.a) )
print("Hid_bias final learn : " + str(rbm.b) )


# Generate new sampling using the trained RBM 
gen_samp = rbm.generate_sample(num_samples = 10)
print("Generated samples: ")

if binary:
    print(np.round(gen_samp))
    
else :
    print(gen_samp)



Qubit-0 probs: [0.8801223 0.1198777]
Qubit-1 probs: [0.8801223 0.1198777]
Qubit-2 probs: [0.8801223 0.1198777]
Qubit-3 probs: [0.8801223 0.1198777]
Qubit-4 probs: [9.99977214e-01 2.27858191e-05]
Qubit-5 probs: [9.99999904e-01 9.62321215e-08]
Qubit-6 probs: [9.99958247e-01 4.17528261e-05]
Qubit-7 probs: [9.99994067e-01 5.93335863e-06]
iq in state probs :  7
Visible : 0.0 0.0
iq in state probs :  6
Visible : 0.0 0.0
iq in state probs :  5
Hidden : 0.0 0.0
iq in state probs :  4
iq in state probs :  3
iq in state probs :  2
iq in state probs :  1
iq in state probs :  0
Qubit-0 probs: [0.8801223 0.1198777]
Qubit-1 probs: [0.8801223 0.1198777]
Qubit-2 probs: [0.8801223 0.1198777]
Qubit-3 probs: [0.8801223 0.1198777]
Qubit-4 probs: [9.99977214e-01 2.27858191e-05]
Qubit-5 probs: [9.99999904e-01 9.62321215e-08]
Qubit-6 probs: [9.99958247e-01 4.17528261e-05]
Qubit-7 probs: [9.99994067e-01 5.93335863e-06]
iq in state probs :  7
Visible : 0.0 0.0
iq in state probs :  6
Visible : 0.0 0.0
iq in sta

In [31]:
W final learn : [[  0.83622353 -10.48497753]
 [-10.4313197    0.40743926]]
Vis_bias final learn : [4.71036486 4.55825946]
Hid_bias final learn : [4.87996752 4.76531972]
Generated samples: 
[[1. 0.]
 [1. 1.]
 [0. 1.]
 [0. 0.]
 [1. 0.]
 [1. 0.]
 [0. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]

-0.012565826271677127